# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-11-04 08:43:29] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-04 08:43:29] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-04 08:43:29] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-04 08:43:29] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


I1104 08:43:31.753046 3038182 transfer_engine.cpp:486] Metrics reporting is disabled (set MC_TE_METRIC=1 to enable)


...

Capturing batches (bs=120 avail_mem=74.27 GB):   5%|▌         | 1/20 [00:00<00:03,  5.35it/s]

Capturing batches (bs=80 avail_mem=74.24 GB):  20%|██        | 4/20 [00:00<00:01, 15.06it/s]

Capturing batches (bs=32 avail_mem=74.21 GB):  50%|█████     | 10/20 [00:00<00:00, 21.42it/s]

Capturing batches (bs=4 avail_mem=74.19 GB):  80%|████████  | 16/20 [00:00<00:00, 21.65it/s]

Capturing batches (bs=1 avail_mem=74.18 GB): 100%|██████████| 20/20 [00:00<00:00, 21.33it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Lisa. I'm a high school student. I love to travel. I have visited many countries in my life. At first, I traveled in Italy. I like the Italian people and I like Italian food. Then, I traveled to Germany. I enjoy the German language and the German people. Now, I have visited many other countries. I like the weather in different places. I like the people there. My favorite country is Brazil. My parents took me to Brazil last year. I love the warm weather. I love the beaches. I like the warm sunlight and the beautiful countryside. I also like the games in Brazil. They are
Prompt: The president of the United States is
Generated text:  married to a woman who has never married before. The president has three children: one son, two daughters, and a grandchild. How many grandchildren does the president have? The president has three children, one son and two daughters. Since the president is not married to any of the daughters, they are all grandchildr

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I am a [Age] year old [Gender] [Occupation]. I am a [Occupation] who has always been [Positive Traits] and [Negative Traits]. I am [Positive Traits] and [Negative Traits] and I am [Positive Traits] and [Negative Traits]. I am [Positive Traits] and [Negative Traits] and I am [Positive Traits] and [Negative Traits]. I am [Positive Traits] and [Negative Traits] and I am [Positive Traits] and [Negative Traits]. I am [Positive Traits] and [Negative Traits] and I am [Positive Traits] and [

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also a major cultural and economic center, hosting numerous museums, theaters, and festivals throughout the year. Paris is a popular tourist destination, attracting millions of visitors each year. The city is also home to many notable French artists, writers, and musicians. The French language is also spoken in Paris, and the city is known for its cuisine, including its famous croissants and pastries. Paris is a vibrant and diverse city with a rich history and culture that continues to captivate visitors from around the

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends that are expected to shape the future of AI:

1. Increased use of AI in healthcare: AI is already being used in healthcare to improve patient outcomes and reduce costs. As AI technology continues to improve, we can expect to see even more widespread use of AI in healthcare, with more sophisticated algorithms and machine learning techniques being developed to improve diagnosis, treatment, and patient care.

2. Increased use of AI in finance: AI is already being used in finance to improve risk management, fraud



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name], and I'm a [职业] with [工作经验] years of experience in [行业] and [技能]. In my [职业] role, I've gained a great deal of knowledge and experience through [实习/工作经历] and have [proven achievements], which I'm confident in sharing with you. So, what do you think? How can I be a good friend to you? [Your Name] is a selfless, hardworking, and dedicated individual with a passion for creating great things. Despite facing obstacles and challenges, [Your Name] is always determined to succeed and achieve his/her goals. If

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.
Paris is the capital city of France and the largest city in Europe by population. It is also the seat of government for the French Republic, and has a rich history dating back over 2,000 years. The city is known for its world-renowned architectur

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 as

 in

 healthcare

 or

 social

In [6]:
llm.shutdown()